In [ ]:
模型融合方式任意，并结合Task-5给出你的最优结果
例如 stacking融合，用你目前评分最高的模型作为基准模型，和其它模型进行stacking整合，得到最终模型及评分结果


In [359]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#导入处理后的数据，4754*81
data = pd.read_csv('data/data_81col.csv',index_col=0)

In [365]:
# 生成训练集和测试集
from sklearn.model_selection import train_test_split
y = data['status']
X = data.drop(['status'],axis = 1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2018)

In [361]:
#导入算法
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#导入模型评估分数
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [367]:
from sklearn.model_selection import KFold

#将训练集分五折
kf = KFold(n_splits=5,shuffle=False)

def get_new(clf,X_train,y_train,X_test):
    """"
    clf  模型
    X_train   3327*80
    y_train 
    X_test  1427*80
    """
    return_train= np.zeros((X_train.shape[0],))   # (3327,)
    return_test_predict = np.zeros((X_test.shape[0],5)) # (1427,5)

    #将 X_train 分成5折，train_index 为4折的索引，用来训练  val_index为1折的索引 用来预测
    for i,(train_index,val_index) in enumerate(kf.split(X_train)):  #train 2661   val 666
    #拟合4折样本
#         clf.fit(X_train.iloc[train_index],y_train.iloc[train_index])  
        clf.fit(X_train[train_index],y_train[train_index])  
        #预测1折的样本并放入 return_train中
        return_train[val_index] = clf.predict(X_train[val_index])

        #预测 X_test 
        return_test_predict[:,i] = clf.predict(X_test)

    return_test = np.mean(return_test_predict,axis=1).reshape(-1,1)  
#     print(return_train.shape)
#     print(return_test.shape)
    #这个模型的5折预测后生成的结果
    return (return_train,return_test)

In [366]:
#对X_train y_train 的索引进行处理, 去掉原来的，改为顺序索引
X_train = X_train.reset_index(drop=True).values
y_train = y_train.reset_index(drop=True).values
X_test = X_test.values

In [368]:
#STACKING
clf = DecisionTreeClassifier(class_weight='balanced', criterion='entropy', max_depth= 5)
dt_train,dt_test = get_new(clf,X_train,y_train,X_test)

clf = RandomForestClassifier(class_weight='balanced', criterion='gini', max_depth=5, n_estimators=50, n_jobs=2)
rf_train,rf_test = get_new(clf,X_train,y_train,X_test)

clf = XGBClassifier(learing_rate=0.001, max_depth=3, n_estimators=50, n_jobs=2)
xgb_train,xgb_test = get_new(clf,X_train,y_train,X_test)


#特征标准化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

clf = LogisticRegression(C=0.1, max_iter=50, solver='liblinear', class_weight='balanced',n_jobs=2)
lr_train,lr_test = get_new(clf,X_train_scaled,y_train,X_test_scaled)

clf = SVC(C=1, kernel='linear', class_weight='balanced',probability=True)
svc_train,svc_test = get_new(clf,X_train_scaled,y_train,X_test_scaled)


In [387]:
#将预测的结果生成一个DataFrame
df_X_train = pd.DataFrame({'dt':dt_train,'rf':rf_train,'xgb':xgb_train,'lr':lr_train,'svc':svc_train})
df_X_test = pd.DataFrame({'dt':dt_test.reshape(-1),'rf':rf_test.reshape(-1),'xgb':xgb_test.reshape(-1),'lr':lr_test.reshape(-1),'svc':svc_test.reshape(-1)})


In [388]:
#使用逻辑回归对结果进行拟合
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(df_X_train,y_train)

y_predict = lr.predict(df_X_test)
print('final accuracy :',accuracy_score(y_test,y_predict))


final accuracy : 0.7918710581639804
